In [1]:
import numpy as np

def fft(x):
    """
    Standard FFT using numpy.
    """
    x = np.asarray(x, dtype=np.complex64)
    return np.fft.fft(x)

def fft_serial(x):
    """
    Serial FFT implementation (Cooley-Tukey radix-2 decimation-in-time).
    Assumes len(x) is power of 2.
    """
    x = np.asarray(x, dtype=np.complex64)
    N = len(x)
    levels = int(np.log2(N))

    # Bit-reversal permutation
    def bit_reverse(n, bits):
        rev = 0
        for _ in range(bits):
            rev = (rev << 1) | (n & 1)
            n >>= 1
        return rev

    reordered = np.zeros(N, dtype=np.complex64)
    for i in range(N):
        j = bit_reverse(i, levels)
        reordered[j] = x[i]

    # Butterfly
    half_size = 1
    while half_size < N:
        phase_shift_step = np.exp(-2j * np.pi / (2 * half_size)).astype(np.complex64)
        for k in range(0, N, 2 * half_size):
            phase = 1.0 + 0.0j
            for n in range(half_size):
                i = k + n
                j = i + half_size
                temp = phase * reordered[j]
                reordered[j] = reordered[i] - temp
                reordered[i] = reordered[i] + temp
                phase *= phase_shift_step
        half_size *= 2

    return reordered


In [2]:
x = np.random.rand(8).astype(np.float16)  # 8-point real input
ref = fft(x)
ser = fft_serial(x)

print("FFT (numpy):", np.round(ref, 4))
print("FFT (serial):", np.round(ser, 4))
print("Same result?", np.allclose(ref, ser, atol=1e-3))


FFT (numpy): [ 3.4722+0.j     -0.2765+0.0925j -0.926 -0.26j   -0.3694+0.1482j
  0.4122+0.j     -0.3694-0.1482j -0.926 +0.26j   -0.2765-0.0925j]
FFT (serial): [ 3.4722+0.j     -0.2765+0.0925j -0.926 -0.26j   -0.3694+0.1482j
  0.4122+0.j     -0.3694-0.1482j -0.926 +0.26j   -0.2765-0.0925j]
Same result? True
